In [133]:
import sqlite3
import pandas as pd
import os
import pandasql

In [134]:
conn = sqlite3.connect("chinook.db")


In [135]:
def allrowsSelect(conn,query):
    cursor = conn.execute(query)
    for row in cursor:
        print(row)

In [136]:
#q1
# ('Edinburgh ', 7)
query='''
SELECT BillingCity,
COUNT(INVOICEID)
FROM invoices
where BILLINGCOUNTRY = 'United Kingdom'
GROUP BY BILLINGCity
ORDER BY  COUNT(INVOICEID) DESC
Limit 1
offset 1

'''
allrowsSelect(conn,query)

('Edinburgh ', 7)


In [137]:
# q2 what is the average numbers of tracks purchased per customer?
# 
# (37.97,)
query='''

select round((Quantity*1.0/num_customers),2)

From(
SELECT  (
    SELECT count(CustomerId)
    FROM   customers
) AS num_customers,
(
    SELECT Sum(Quantity)
    FROM   invoice_items
) AS  Quantity
)
'''
allrowsSelect(conn,query)

(37.97,)


In [138]:
#q3 How many albums have jazz tracks? 
# join genres tracks albums , genres ID 
# ('Jazz', 130)

query='''
select g.Name, COUNT (a.AlbumId)
FROM albums a
inner join tracks t on t.AlbumId=a.AlbumId
inner join genres g on t.GenreId=g.GenreId
Where g.Name="Jazz"



'''
allrowsSelect(conn,query)

('Jazz', 130)


In [139]:
#q4. How many comedy tracks were purchased in total? (Note you can ignore the Quantity  field in the invoice_items table which is always one.) 
# join the genres, tracks,(TrackId)  invoice_items (TrackId)

# Answer  :9
query='''


select  COUNT(it.Quantity)
FROM invoice_items it
INNER JOIN  tracks t on T.TrackId=it.TrackId
INNER JOIN Genres g on t.GenreId=g.GenreId
WHERE g.Name="Comedy"



'''
allrowsSelect(conn,query)


(9,)


In [140]:
#q5 what is the name of customer who purchesed the most Blues tracks?


# 'Leonie', 'Köhler',9
query='''

select c.FirstName, c.LastName, COUNT(i.CustomerId)
FROM customers c 
INNER JOIN invoices i ON c.CustomerId=i.CustomerId
INNER JOIN invoice_items il on i.InvoiceId= il.InvoiceId
INNER JOIN Tracks t on il.TrackId=t.TrackId
INNER JOIN Genres g on t.GenreId=g.GenreId
WHERE g.Name="Blues"
GROUP BY i.CustomerId,c.FirstName,c.LastName
ORDER BY COUNT(i.CustomerId) DESC
LIMIT 1



'''
allrowsSelect(conn,query)

('Leonie', 'Köhler', 9)


In [141]:
# q6 On the Grunge playlist, what is the name of the track that comes second  alphabetically? 

# ('Black Hole Sun',)
query='''
select t.Name
from playlists p
Inner join playlist_track pt on p.PlaylistId=pt.PlaylistId
Inner join tracks t on t.TrackId=pt.TrackId
where p.Name='Grunge'
order by t.Name
limit 1
offset 1


'''
allrowsSelect(conn,query)

('Black Hole Sun',)


In [142]:
#q7 

# (10,)
query='''
select Count(*)
From(select Y.AlbumId, Sum(TOTAL) As Grand_total
From(SELECT X.AlbumId, X.UnitPrice* X.Quantity AS TOTAL
From (SELECT  a.AlbumId,il.UnitPrice, il.Quantity 
FROM albums a 
Inner join tracks t on t.AlbumId=a.AlbumId
Inner join invoice_items il on il.TrackId=t.TrackId
)As X)As Y

Group by Y.AlbumId
Having Grand_total>20)


'''
allrowsSelect(conn,query)

(10,)


In [143]:
# q8
# 19.7
query='''
select round(tot_customers*1.0/tot_supp,1)
from(
SELECT  (
    SELECT Count(*)
    FROM   customers
) AS tot_customers,
(
    SELECT  SupportRepId
    FROM   customers
    group by SupportRepId
) AS tot_supp
)
'''

allrowsSelect(conn,query)



(19.7,)


In [144]:
# q9  How many genres had sales exceeding $200? 
# join genres, tracks (GenreId),unit price
# answer 4
query='''
select count(*) 
from (
select X.GenreId, sum(Total)
From(select g.GenreId,t.UnitPrice As Total
From genres g 
Inner Join tracks t on g.GenreId=t.GenreId)As X
Group By X.GenreId
Having sum(Total)>200)
'''

allrowsSelect(conn,query)

(4,)


In [145]:
#q10  What artist has the third most albums in the database? 
# join artists ArtistId, albums(ArtistId)
# answer 'Deep Purple', 11

query='''

select Ar.Name,Count(Al.
AlbumId)

from artists Ar 
Inner join albums Al on Ar.ArtistId=Al.ArtistId
Group By Ar.ArtistId
order by Count(Al.
AlbumId) desc
Limit 1
offset 2


'''

allrowsSelect(conn,query)

('Deep Purple', 11)


In [146]:
# q11Find the names of the tracks on the album "Fireball" and put them in alphabetical  order. What is the name of the third track (in alphabetical order, not in the order in which  they appeared on the album)? 
# answer ： ('Fools',)

query='''
select t.Name
From tracks t
inner join albums AL  on t.AlbumId=AL.AlbumId
group by t.Name
Having AL.Title="Fireball"
order by t.Name ASC
limit 1
offset 2



'''
allrowsSelect(conn,query)

('Fools',)


In [147]:
# q12 How many artists in the database do not have any albums in the database?
# inner join artists,albums
# Answer:71


query='''
select Count(art.ArtistId)
From artists art
Left  join albums al on art.ArtistId= al.ArtistId
where al.AlbumId IS NULL

'''
allrowsSelect(conn,query)


(71,)


In [130]:
# q13
# Answer:("Monteverdi: L'Orfeo", 66639.0)
query='''

select al.title, total(t.Milliseconds) 
from albums al 
Inner join tracks t on t.AlbumId=al.AlbumId
group by al.AlbumId 
order by total(t.Milliseconds) 
Limit 1
offset 1


'''
allrowsSelect(conn,query)

("Monteverdi: L'Orfeo", 66639.0)


In [131]:
# q14
#Find the number of invoices per billing city. Although there are 53 billing cities in the  invoices table, there are are only a few distinct values of invoices per city. What are all  of these values?
#
query='''
select BillingCity, count(*) as billing_city
from invoices
group by BillingCity

'''
allrowsSelect(conn,query)

('Amsterdam', 7)
('Bangalore', 6)
('Berlin', 14)
('Bordeaux', 7)
('Boston', 7)
('Brasília', 7)
('Brussels', 7)
('Budapest', 7)
('Buenos Aires', 7)
('Chicago', 7)
('Copenhagen', 7)
('Cupertino', 7)
('Delhi', 7)
('Dijon', 7)
('Dublin', 7)
('Edinburgh ', 7)
('Edmonton', 7)
('Fort Worth', 7)
('Frankfurt', 7)
('Halifax', 7)
('Helsinki', 7)
('Lisbon', 7)
('London', 14)
('Lyon', 7)
('Madison', 7)
('Madrid', 7)
('Montréal', 7)
('Mountain View', 14)
('New York', 7)
('Orlando', 7)
('Oslo', 7)
('Ottawa', 7)
('Paris', 14)
('Porto', 7)
('Prague', 14)
('Redmond', 7)
('Reno', 7)
('Rio de Janeiro', 7)
('Rome', 7)
('Salt Lake City', 7)
('Santiago', 7)
('Sidney', 7)
('Stockholm', 7)
('Stuttgart', 7)
('São José dos Campos', 7)
('São Paulo', 14)
('Toronto', 7)
('Tucson', 7)
('Vancouver', 7)
('Vienne', 7)
('Warsaw', 7)
('Winnipeg', 7)
('Yellowknife', 7)


In [132]:
#q14
# Ans: 7, 6, 14
''''''
query='''
select distinct bil
from (select BillingCity, count(*)  As bil
      from invoices
      group by BillingCity)
'''
allrowsSelect(conn,query)

(7,)
(6,)
(14,)
